<a href="https://colab.research.google.com/gist/ruvnet/c8855ccb2315fb6d5c773e6618350058/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```markdown
# Deploying Multi-Step Serverless Applications with AWS SAM

The AWS Serverless Application Model (SAM) is an open-source framework for building serverless applications on AWS. It provides a simplified way to define and deploy serverless resources like Lambda functions, API Gateway APIs, Step Functions state machines, DynamoDB tables and more using concise declarative YAML templates[2][5].

## Development Workflow

1. **Initialize Project**: Use the SAM CLI to initialize a new serverless project from a quick start template[1][13]. This generates the scaffolding for your application with a sample SAM template.

2. **Define Architecture**: Define your application architecture in the SAM template using the simplified YAML syntax. This includes Lambda functions, APIs, databases, event source mappings, Step Functions state machines etc.[2]

3. **Develop Locally**: Develop your application code locally. Use the SAM CLI to build your Lambda function code, run and debug it locally to speed up the development feedback loop[5][10].

4. **Package Application**: Once ready, use the SAM CLI to package your application. This zips up your code artifacts and uploads them to S3[1][13].

5. **Deploy Application**: Deploy the packaged application using the SAM CLI. This transforms the SAM template into a CloudFormation stack and deploys the defined resources[1][17].

## Multi-Step Workflows

For multi-step workflows, AWS Step Functions can be defined directly in the SAM template to orchestrate complex workflows involving multiple Lambda functions and other services[1][19]. Step Functions supports defining the workflow as an Amazon States Language JSON definition that gets deployed along with the Lambda functions.

## Best Practices

- **Modular Architecture**: Decompose into multiple SAM templates/repositories aligned to microservices or functional boundaries. Avoid large monolithic applications.

- **Nested Applications**: Use nested applications to compose together multiple SAM templates as modular building blocks. The root stack can reference child stacks[8].

- **Separate Environments**: Use AWS Organizations to provision separate development, staging and production AWS accounts. Give developers their own accounts to safely experiment[16].

- **Safe Deployments**: Implement safe deployments using canary deployments and CodeDeploy hooks in SAM templates to gradually shift traffic to new versions[16].

- **Event-Driven Architecture**: Use event-driven patterns to decouple services. Services publish events to EventBridge which triggers downstream services[12].

This allows building serverless applications with some autonomy, where individual components can be independently developed, tested and deployed as loosely coupled microservices. Step Functions provides a way to coordinate across services for multi-step workflows.

SAM provides guard rails and best practices while still allowing flexibility to build non-trivial serverless applications. The simplified development experience allows quickly going from idea to production[5].

```

In [ ]:
# Install dependencies
!pip install aws-sam-cli dspy ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create text input widgets for AWS credentials
access_key_input = widgets.Text(
    placeholder='Enter your AWS Access Key ID',
    description='Access Key ID:',
    disabled=False
)
secret_key_input = widgets.Text(
    placeholder='Enter your AWS Secret Access Key',
    description='Secret Access Key:',
    disabled=False
)

# Display the input widgets
display(access_key_input, secret_key_input)

# Set the entered AWS credentials as environment variables
%env AWS_ACCESS_KEY_ID={access_key_input.value}
%env AWS_SECRET_ACCESS_KEY={secret_key_input.value}

In [ ]:
# Define global variables
global sam_template
global dspy_agent_code

# Define the SAM template
sam_template = '''
AWSTemplateFormatVersion: '2010-09-09'
Transform: AWS::Serverless-2016-10-31
Description: DSPy Agent Application

Resources:
  DataBucket:
    Type: AWS::S3::Bucket

  DSPyAgentFunction:
    Type: AWS::Serverless::Function
    Properties:
      CodeUri: dspy-agent/
      Handler: app.lambda_handler
      Runtime: python3.9
      Environment:
        Variables:
          DATA_BUCKET: !Ref DataBucket
'''

# Define the DSPy agent code
dspy_agent_code = '''
import os
import dspy

def lambda_handler(event, context):
    data_bucket = os.environ['DATA_BUCKET']
    data = dspy.load_data(f's3://{data_bucket}/data.csv')

    model = dspy.train_model(data)

    # Perform any other tasks with the trained model
    # ...

    return {
        'statusCode': 200,
        'body': 'DSPy agent executed successfully'
    }
'''

In [ ]:
# Write the SAM template to a file
with open('sam-app.yaml', 'w') as file:
    file.write(sam_template)

# Write the DSPy agent code to a file
os.makedirs('dspy-agent', exist_ok=True)
with open('dspy-agent/app.py', 'w') as file:
    file.write(dspy_agent_code)

In [ ]:
# Package the application
!sam package --template-file sam-app.yaml --output-template-file packaged.yaml --s3-bucket YOUR_BUCKET_NAME

In [ ]:
# Deploy the packaged application
!sam deploy --template-file packaged.yaml --stack-name dspy-agent-app --capabilities CAPABILITY_IAM